# Sentiment Analysis of Financial Data Using Hugging Face NLP Models

This notebook provides an introduction for model developers on how to document a natural language processing (NLP) model using the ValidMind Developer Framework. It shows you how to set up the ValidMind Developer Framework, initialize the client library, and load the dataset, followed by performing a sentiment analysis of financial news data using several different Hugging Face transformers. 
<!--- NR TO DO This differs from the dataset being downloaded later on
The use case is a sentiment analysis of financial news data (https://huggingface.co/datasets/financial_phrasebank).--->

- Initializing the ValidMind Developer Framework
- Running a test various tests to quickly generate documentation about the data and model

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, we recommend going through the following resources first:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [1]:
%pip install -q validmind


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook.

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Hugging Face - Text Sentiment Analysis`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace the code below with the code snippet from your project ##


import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)
  

2024-01-19 12:44:39,511 - INFO(validmind.api_client): Connected to ValidMind. Project: [Demo] Hugging Face - Text Sentiment Analysis - Initial Validation (clr4sqrbq0047uqy6o3uy3o2u)


### Preview the documentation template

A template predefines sections for your documentation project and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [3]:
vm.preview_template()

Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), Accordion(children=(HTML(value='<p…

### Download the test dataset

To perform the sentiment analysis for financial news, you need a sample dataset:  

<!--- NR TO DO Is this the wrong data set? It's different from what the intro mentions --->
1. Download the sample dataset from https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news.
   
   This dataset contains two columns, `Sentiment` and `News Headline`. The sentiment can be `negative`, `neutral` or `positive`.

2. Move the CSV file that contains the dataset into the current directory. 

In [4]:
import pandas as pd

df = pd.read_csv('./datasets/sentiments.csv')
sample = df.sample(10)
sample

,Sentiment,Sentence
2634,neutral,"The fish content of the nuggets is 85 % , and ..."
4274,neutral,The resignation will be in effect immediately .
4267,neutral,The new apartment block is going up very close...
3226,neutral,The CVs of the proposed members are available ...
1167,neutral,"Also , a six-year historic analysis is provide..."
2510,neutral,Metso 's delivery will include a complete coat...
4133,neutral,"The effect may remain short-lived , however ."
4762,negative,Sales by Seppala diminished by 6 per cent .
985,positive,"The launch of the plant in June went well , an..."
3887,neutral,The plant is scheduled for completion in late ...


## NLP data quality tests

Before we proceed with the analysis, it's crucial to ensure the quality of our NLP data. We can run the "data preparation" section of the template to validate the data's integrity and suitability.

In [16]:
vm_ds = vm.init_dataset(
    dataset=df,
    text_column='Sentence',
    target_column="Sentiment"
)

text_data_test_plan = vm.run_documentation_tests(
    section="data_preparation", 
    inputs = {"dataset":vm_ds}
)

2024-01-19 12:45:59,778 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juanvalidmind/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juanvalidmind/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Hugging Face transformers

### 1. Hugging Face: FinancialBERT for Sentiment Analysis

Let's now explore integrating and testing FinancialBERT (https://huggingface.co/ahmedrachid/FinancialBERT-Sentiment-Analysis
), a model designed specifically for sentiment analysis in the financial domain:

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
hfmodel = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

#### Initialize the ValidMind dataset

In [7]:
# Load a test dataset with 100 rows only
vm_test_ds = vm.init_dataset(
    dataset=df.head(100),
    text_column="Sentence",
    target_column="Sentiment",
)

2024-01-19 12:44:49,336 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


#### Initialize the ValidMind model

When initializing a ValidMind model, we pre-calculate predictions on the test dataset. This operation can take a long time for large datasets.

In [8]:
vm_model_1 = vm.init_model(
    hfmodel,
    test_ds=vm_test_ds,
)

#### Run model validation tests

It's possible to run a subset of tests on the documentation template by passing a `section` parameter to `run_documentation_tests()`. Let's run the tests that correspond to model validation only:

In [17]:
full_suite = vm.run_documentation_tests(
    section="model_development",
    inputs = {
        "dataset":vm_test_ds,
        "model":vm_model_1
    }
)

### 2. Hugging Face: distilRoberta-financial-sentiment 

The distilRoberta-financial-sentiment model (https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis) was fine-tuned on the phrasebank dataset: https://huggingface.co/datasets/financial_phrasebank.

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
hfmodel = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

#### Initialize the ValidMind model

In [11]:
vm_model_2 = vm.init_model(
    hfmodel,
    test_ds=vm_test_ds,
)

In [18]:
full_suite = vm.run_documentation_tests(
    section="model_development",
    inputs = {
        "dataset":vm_test_ds,
        "model":vm_model_2,
        "models":[vm_model_1]
    }
)

### 3. Hugging Face: Financial-RoBERTa

The Financial-RoBERTa model (https://huggingface.co/soleimanian/financial-roberta-large-sentiment) is another financial sentiment analysis model trained on large amounts of data including:

- Financial Statements
- Earnings Announcements
- Earnings Call Transcripts
- Corporate Social Responsibility (CSR) Reports
- Environmental, Social, and Governance (ESG) News
- Financial News
- Etc.

#### Load the model directly

In [13]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("soleimanian/financial-roberta-large-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("soleimanian/financial-roberta-large-sentiment")
hfmodel = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [14]:
vm_model_3 = vm.init_model(
    hfmodel,
    test_ds=vm_test_ds,
)

In [19]:
full_suite = vm.run_documentation_tests(
    section="model_development",
    inputs = {
        "dataset":vm_test_ds,
        "model":vm_model_3,
        "models":[vm_model_1, vm_model_2]
    }
)

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the prompt validation test results as part of your model documentation right in the ValidMind Platform UI: 

1. Log back into the [Platform UI](https://app.prod.validmind.ai) 

2. Go to **Documentation Projects** > **YOUR_DOCUMENTATION_PROJECT** > **Documentation**.

3. Expand **2. Data Preparation** or **3. Model Development** to review all test results.

What you can see now is a more easily consumable version of the prompt validation testing you just performed, along with other parts of your documentation project that still need to be completed. 

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).

